<a href="https://colab.research.google.com/github/aadityane93/SortingLine-ml/blob/main/Feature_Selection_and_Model_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install tsfresh
from google.colab import files
import numpy as np
import pandas as pd
from tsfresh import extract_features, select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_selection.relevance import calculate_relevance_table
from sklearn.model_selection import train_test_split, KFold, cross_val_predict, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score, precision_score
import xgboost as xgb

In [3]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
folder_id = '15myxitNBue9FKPBvNm-j_FS_fiElz3C8' # files

folder_list = drive.ListFile({'q': "'{}' in parents and mimeType='application/vnd.google-apps.folder' and trashed=false".format(folder_id)}).GetList()


for folder in folder_list:
    folder_title = folder['title']
    print(f"Processing folder: {folder_title}")
    file_list = drive.ListFile({'q': "'{}' in parents and trashed=false".format(folder['id'])}).GetList()

    for index, file in enumerate(file_list):
        file_title = file['title']
        print(f"Downloading {file_title}")
        file.GetContentFile(file_title)

Processing folder: 1000 files
Processing folder: 100 files
Processing folder: 10 files


In [6]:
def read_file(filename):
  return pd.read_csv(filename, sep=',')

extracted_train_feature = pd.DataFrame(read_file('extracted-100hz-train-feature-50percent-100files.csv'))
extracted_test_feature = pd.DataFrame(read_file('extracted-100hz-test-feature-50percent-100files.csv'))
extracted_train_target = pd.DataFrame(read_file('extracted-100hz-train-target-50percent-100files.csv'))
extracted_test_target = pd.DataFrame(read_file('extracted-100hz-test-target-50percent-100files.csv'))

In [9]:
extracted_train_target = extracted_train_target.drop(columns=['Unnamed: 0'])
extracted_test_target = extracted_test_target.drop(columns=['Unnamed: 0'])
extracted_train_feature = extracted_train_feature.drop(columns=['Unnamed: 0'])
extracted_test_feature = extracted_test_feature.drop(columns=['Unnamed: 0'])

In [27]:
extracted_train_feature.head()

,I_In__variance_larger_than_standard_deviation,I_In__has_duplicate_max,I_In__has_duplicate_min,I_In__has_duplicate,I_In__sum_values,I_In__abs_energy,I_In__mean_abs_change,I_In__mean_change,I_In__mean_second_derivative_central,I_In__median,...,Temp(°C)__fourier_entropy__bins_5,Temp(°C)__fourier_entropy__bins_10,Temp(°C)__fourier_entropy__bins_100,Temp(°C)__permutation_entropy__dimension_3__tau_1,Temp(°C)__permutation_entropy__dimension_4__tau_1,Temp(°C)__permutation_entropy__dimension_5__tau_1,Temp(°C)__permutation_entropy__dimension_6__tau_1,Temp(°C)__permutation_entropy__dimension_7__tau_1,Temp(°C)__query_similarity_count__query_None__threshold_0.0,Temp(°C)__mean_n_absolute_max__number_of_maxima_7
0,0.0,0.0,1.0,1.0,189.99,60.2121,0.042299,0.000305,-8.051530e-06,0.29,...,1.204209,1.795131,3.805391,1.318114,2.152424,2.987462,3.746941,4.391745,0.0,37.88
1,0.0,0.0,1.0,1.0,185.58,59.6738,0.042040,0.000216,0.000000e+00,0.29,...,1.436031,2.089244,4.045644,1.410305,2.314589,3.235639,4.075079,4.799483,0.0,37.63
2,0.0,0.0,1.0,1.0,184.09,59.0125,0.043344,0.000250,-4.625929e-20,0.30,...,1.139872,1.786103,3.792321,1.462869,2.450789,3.414714,4.310813,5.059538,0.0,37.63
3,0.0,0.0,0.0,1.0,187.40,60.1416,0.042311,0.000246,-1.642036e-05,0.29,...,1.219551,1.839760,3.856714,1.485377,2.524549,3.559655,4.512030,5.219138,0.0,37.63
4,0.0,0.0,1.0,1.0,183.78,59.4834,0.041930,0.000285,1.680672e-05,0.29,...,1.211701,1.898553,3.854095,1.467557,2.479623,3.458075,4.381721,5.129161,0.0,37.63


In [22]:
selected_features_train = select_features(extracted_train_feature, extracted_train_target['color'])

In [24]:
selected_features_test = extracted_test_feature[selected_features_train.columns]

In [25]:
selected_features_train.head()

,I_Out(A)__sum_of_reoccurring_data_points,"I_In__fft_aggregated__aggtype_""variance""","I_Out(A)__fft_coefficient__attr_""angle""__coeff_1",I_In__sum_of_reoccurring_data_points,"Temp(°C)__fft_coefficient__attr_""abs""__coeff_0","Temp(°C)__fft_coefficient__attr_""real""__coeff_0",Temp(°C)__abs_energy,Temp(°C)__sum_values,Temp(°C)__sum_of_reoccurring_data_points,I_Out(A)__range_count__max_1__min_-1,...,"I_Out(A)__fft_coefficient__attr_""abs""__coeff_52","I_In__fft_coefficient__attr_""abs""__coeff_18","I_In__fft_coefficient__attr_""abs""__coeff_49","Temp(°C)__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.4","I_Out(A)__fft_coefficient__attr_""abs""__coeff_75","Temp(°C)__change_quantiles__f_agg_""mean""__isabs_True__qh_0.4__ql_0.2","Temp(°C)__change_quantiles__f_agg_""var""__isabs_False__qh_0.4__ql_0.2","Temp(°C)__change_quantiles__f_agg_""var""__isabs_True__qh_0.4__ql_0.2",I_Out(A)__partial_autocorrelation__lag_4,"I_Out(A)__linear_trend__attr_""pvalue"""
0,189.20,10023.276497,12.281726,186.58,23397.99,23397.99,878768.6987,23397.99,23397.99,622.0,...,1.936640,2.818966,0.864501,0.009772,1.365618,0.105332,0.026331,0.015238,0.313336,9.732668e-12
1,183.39,8893.805173,13.918459,182.01,22658.52,22658.52,850026.7576,22658.52,22658.52,602.0,...,1.709872,2.444789,2.194649,0.115036,1.655305,0.000000,0.000000,0.000000,0.190863,2.882170e-12
2,182.81,9239.818040,13.999959,181.48,22567.51,22567.51,846013.1913,22567.51,22567.51,601.0,...,1.248486,2.373812,0.837620,0.122736,1.623350,0.000000,0.000000,0.000000,0.228127,1.075370e-11
3,183.34,9143.044817,14.953357,183.44,22888.18,22888.18,857410.4134,22888.18,22888.18,610.0,...,1.589452,2.286361,1.707862,0.120370,1.456777,0.000000,0.000000,0.000000,0.264175,1.424840e-12
4,182.58,8209.689272,11.591896,178.62,22371.61,22371.61,838352.7793,22371.61,22371.61,596.0,...,1.036741,2.623803,2.911375,0.122650,1.204775,0.000000,0.000000,0.000000,0.233434,1.172800e-09


In [26]:
selected_features_test.head()

,I_Out(A)__sum_of_reoccurring_data_points,"I_In__fft_aggregated__aggtype_""variance""","I_Out(A)__fft_coefficient__attr_""angle""__coeff_1",I_In__sum_of_reoccurring_data_points,"Temp(°C)__fft_coefficient__attr_""abs""__coeff_0","Temp(°C)__fft_coefficient__attr_""real""__coeff_0",Temp(°C)__abs_energy,Temp(°C)__sum_values,Temp(°C)__sum_of_reoccurring_data_points,I_Out(A)__range_count__max_1__min_-1,...,"I_Out(A)__fft_coefficient__attr_""abs""__coeff_52","I_In__fft_coefficient__attr_""abs""__coeff_18","I_In__fft_coefficient__attr_""abs""__coeff_49","Temp(°C)__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.4","I_Out(A)__fft_coefficient__attr_""abs""__coeff_75","Temp(°C)__change_quantiles__f_agg_""mean""__isabs_True__qh_0.4__ql_0.2","Temp(°C)__change_quantiles__f_agg_""var""__isabs_False__qh_0.4__ql_0.2","Temp(°C)__change_quantiles__f_agg_""var""__isabs_True__qh_0.4__ql_0.2",I_Out(A)__partial_autocorrelation__lag_4,"I_Out(A)__linear_trend__attr_""pvalue"""
0,194.29,10332.511806,9.374856,192.58,24193.48,24193.48,906088.1374,24193.48,24193.48,644.0,...,2.012123,1.215364,0.932181,0.107080,2.009029,0.000000,0.000000,0.000000,0.208264,4.882469e-12
1,185.65,7804.108885,-2.541205,181.63,22690.78,22690.78,849634.4864,22690.78,22690.78,604.0,...,84.492819,89.877914,89.090469,0.123134,81.017946,0.000000,0.000000,0.000000,-0.000536,1.325328e-01
2,183.82,8041.320629,11.374796,182.05,22536.02,22536.02,840855.5326,22536.02,22536.02,602.0,...,1.561495,2.385978,1.493075,0.000000,2.200444,0.102819,0.025705,0.015133,0.156841,3.599197e-11
3,190.41,10079.223927,14.866742,187.37,23354.38,23354.38,871297.3094,23354.38,23354.38,625.0,...,2.542209,1.989706,1.243267,0.000000,1.799172,0.107200,0.026800,0.015308,0.191705,2.481056e-13
4,193.53,9745.087070,12.739791,191.33,23916.08,23916.08,892330.7854,23916.08,23916.08,639.0,...,2.824340,1.727575,1.689163,0.000000,1.277435,0.103906,0.025977,0.015180,0.137282,1.259866e-12


In [20]:
extracted_train_target

,color
0,0
1,0
2,0
3,0
4,0
...,...
195,3
196,3
197,3
198,3


In [30]:
model = RandomForestClassifier()
model.fit(selected_features_train,extracted_train_target)
y_pred = model.predict(selected_features_test)
accuracy = accuracy_score(extracted_train_target, y_pred)
print(f"Accuracy: {accuracy}")
print(classification_report(extracted_train_target, y_pred))
print(confusion_matrix(extracted_train_target, y_pred))

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Accuracy: 0.855
              precision    recall  f1-score   support

           0       1.00      0.94      0.97        50
           1       0.93      0.52      0.67        50
           2       0.65      1.00      0.79        50
           3       1.00      0.96      0.98        50

    accuracy                           0.85       200
   macro avg       0.89      0.85      0.85       200
weighted avg       0.89      0.85      0.85       200

[[47  0  3  0]
 [ 0 26 24  0]
 [ 0  0 50  0]
 [ 0  2  0 48]]
